In [1]:
import ast
import json
import os
import shutil

import albumentations
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom as dicom
import tensorflow as tf
import tensorflow_addons as tfa
import torch
from IPython.core.interactiveshell import InteractiveShell
from numba import cuda
from PIL import Image
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
from tqdm import tqdm

from src.augment import Aug, Aug_Crop, Aug_No_transform, Flip_Aug
from src.generator import Generator, GetModel

InteractiveShell.ast_node_interactivity = "all"

In [2]:
df = pd.read_csv("/app/_data/predicted_crop_v1.csv")
list_wrong = df[(df["class"] != "negative") & (df["label"] == "none 1 0 0 1 1")][
    "id_image"
].tolist()
df = df.query("id_image not in @list_wrong").reset_index(drop=True)

with open("/app/_data/base_config.json", "r") as f:
    base_config = json.load(f)
base_config["EFFB7"]["SEED"] = 98
base_config

{'EFFB7': {'IMG_SIZE': 600,
  'BATCH_SIZE': 4,
  'SEED': 98,
  'WEIGHTS': '/app/_data/noisy-student-efficientnet-b7/efficientnetb7_notop.h5'},
 'CLASS': {'negative': 0, 'typical': 1, 'indeterminate': 2, 'atypical': 3},
 'EFFB4': {'IMG_SIZE': 380,
  'BATCH_SIZE': 50,
  'SEED': 42,
  'WEIGHTS': '/app/_data/efficientnet-b4_noisy-student_notop.h5'},
 'EFFB0': {'IMG_SIZE': 224,
  'BATCH_SIZE': 120,
  'SEED': 42,
  'WEIGHTS': 'imagenet'},
 'EFFB6': {'IMG_SIZE': 528,
  'BATCH_SIZE': 8,
  'SEED': 42,
  'WEIGHTS': '/app/_data/noisy-student-efficientnet-b6/efficientnetb6_notop.h5'}}

In [3]:
weight_for_negative = (1 / sum(df["class"] == "negative")) * (df.shape[0] / 4.0)
weight_for_typical = (1 / sum(df["class"] == "typical")) * (df.shape[0] / 4.0)
weight_for_indeterminate = (1 / sum(df["class"] == "indeterminate")) * (
    df.shape[0] / 4.0
)
weight_for_atypical = (1 / sum(df["class"] == "atypical")) * (df.shape[0] / 4.0)
non_negative = (1 / sum(df["class"] != "negative")) * (df.shape[0] / 2.0)
negative = (1 / sum(df["class"] == "negative")) * (df.shape[0] / 2.0)
class_weights = {
    "output1": [
        weight_for_negative,
        weight_for_typical,
        weight_for_indeterminate,
        weight_for_atypical,
    ],
    "output2": [negative, non_negative],
}
class_weights
loss_weights = {"output1": 1, "output2": 0.1}

{'output1': [0.9121543778801844,
  0.5266045892916528,
  1.4291516245487366,
  3.278467908902692],
 'output2': [1.8243087557603688, 0.6887777294475859]}

# train

In [4]:
policy = keras.mixed_precision.experimental.Policy("mixed_float16")
keras.mixed_precision.experimental.set_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale


In [5]:
skf = StratifiedKFold(
    n_splits=5, random_state=base_config["EFFB7"]["SEED"], shuffle=True
)
train_ids = []
val_ids = []
for train_index, valid_index in skf.split(df, df["class"]):
    train_ids.append(train_index)
    val_ids.append(valid_index)

In [6]:
for n in range(0, 5):
    print("\n epoch #" + str(n) + "\n")
    val = df.loc[val_ids[n]].sample(frac=1, random_state=base_config["EFFB7"]["SEED"])
    train = df.loc[train_ids[n]].sample(
        frac=1, random_state=base_config["EFFB7"]["SEED"]
    )

    gen_train = Generator(
        df=train,
        batch_size=base_config["EFFB7"]["BATCH_SIZE"],
        seed=base_config["EFFB7"]["SEED"],
        img_size=base_config["EFFB7"]["IMG_SIZE"],
        cache_img_path="/app/_data/crop_npy_600/",
        shuffle=True,
        label_columns=[
            "Negative for Pneumonia",
            "Typical Appearance",
            "Indeterminate Appearance",
            "Atypical Appearance",
        ],
        augment_fn=Aug_No_transform,
        crop=True,
    )
    gen_valid = Generator(
        df=val,
        batch_size=base_config["EFFB7"]["BATCH_SIZE"],
        seed=base_config["EFFB7"]["SEED"],
        img_size=base_config["EFFB7"]["IMG_SIZE"],
        cache_img_path="/app/_data/crop_npy_600/",
        shuffle=False,
        label_columns=[
            "Negative for Pneumonia",
            "Typical Appearance",
            "Indeterminate Appearance",
            "Atypical Appearance",
        ],
        augment_fn=None,
        crop=True,
    )

    get_m = GetModel(
        model_name="EFFB7",
        lr=0.0007,
        activation_func="softmax",
        weights=base_config["EFFB7"]["WEIGHTS"],
        n_classes=4,
        top_dropout_rate=None,
        loss="categorical_crossentropy",
        metrics=["acc"],
    )
    model = get_m.get_model()

    callbacks = get_m.make_callback(
        model_path="/app/_data/models/EffB7_cropped/",
        model_name="EffB7_" + str(n) + ".h5",
        tensorboard_path="/app/.tensorboard/EffB7_cr00_" + str(n),
        patience_ES=12,
        patience_RLR=2,
        factor_LR=0.8,
        metric_for_monitor="val_loss",
        metric_mode="min",
    )

    history = model.fit(
        gen_train,
        validation_data=gen_valid,
        epochs=50,
        steps_per_epoch=len(train) // base_config["EFFB7"]["BATCH_SIZE"],
        validation_steps=len(val) // base_config["EFFB7"]["BATCH_SIZE"],
        verbose=1,
        workers=20,
        max_queue_size=500,
        callbacks=callbacks,
    )
    keras.backend.clear_session()


 epoch #0

  opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt)

Warning! Model path already exists.
Epoch 5/50
1266/1266 [==============================] - 479s 314ms/step - loss: 1.4548 - acc: 0.4025 - val_loss: 2.0591 - val_acc: 0.4581

Epoch 00005: val_loss improved from inf to 2.05906, saving model to /app/_data/models/EffB7_cropped/EffB7_0.h5
Epoch 6/50
1266/1266 [==============================] - 326s 257ms/step - loss: 1.4486 - acc: 0.4037 - val_loss: 1.2887 - val_acc: 0.4755

Epoch 00006: val_loss improved from 2.05906 to 1.28873, saving model to /app/_data/models/EffB7_cropped/EffB7_0.h5
Epoch 7/50
1266/1266 [==============================] - 326s 257ms/step - loss: 1.4733 - acc: 0.4231 - val_loss: nan - val_acc: 0.4842

Epoch 00007: val_loss did not improve from 1.28873
Epoch 8/50
1266/1266 [==============================] - 327s 258ms/step - loss: 1.4538 - acc: 0.4149 - val_loss: nan - val_acc: 0.4628

Epoch 00008: val_loss did not improve from 1.28873

Ep